In [ ]:
import argparse
import os
from flwr.client import ClientApp, NumPyClient
import tensorflow as tf
import numpy as np
import cv2
from sklearn.preprocessing import LabelEncoder

# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Parse arguments
parser = argparse.ArgumentParser(description="Flower")
parser.add_argument(
    "--partition-id",
    type=int,
    choices=[0, 1, 2],
    default=0,
    help="Partition of the dataset (0, 1, or 2). "
    "The dataset is divided into 3 partitions created artificially.",
)
args, _ = parser.parse_known_args()

# Define function to load images and labels
def load_images_and_labels(dataset_folder):
    image_paths = []
    labels = []

    for label in os.listdir(dataset_folder):
        label_folder = os.path.join(dataset_folder, label)
        if os.path.isdir(label_folder):
            for image_file in os.listdir(label_folder):
                image_path = os.path.join(label_folder, image_file)
                image_paths.append(image_path)
                labels.append(label)

    return image_paths, labels

# Load images and labels
dataset_folder = "/Users/adityasoni/Desktop/AugmentedAlzheimerDataset2"
image_paths, labels = load_images_and_labels(dataset_folder)

# Encode labels into integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Load image with labels (for demonstration)
def load_image_with_label(image_path, label):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img, label

# Resize image to fixed dimensions
def resize_image(img, target_size=(224, 224)):
    return cv2.resize(img, target_size)

# Define Flower client
class FlowerClient(NumPyClient):
    def __init__(self):
        super().__init__()

    def get_parameters(self, config=None):
        return []  # Empty list since there are no model parameters to send

    def fit(self, parameters, config=None):
        # You can define your model and training process here
        # For example:
        model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(224, (3, 3), activation='relu', input_shape=(224, 224, 3)),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10, activation='softmax')
        ])
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        
        # Load image data
        images = []
        for image_path, label in zip(image_paths, labels):
            img, _ = load_image_with_label(image_path, label)
            img_resized = resize_image(img)  # Resize image
            images.append(img_resized)
        
        # Convert list to NumPy array
        x_train = np.array(images)
        y_train = np.array(labels)
        
        # Train the model
        model.fit(x_train, y_train, epochs=1, batch_size=32)
        
        # Return the updated model weights
        return model.get_weights(), len(x_train), {}

    def evaluate(self, parameters, config=None):
        # You can define your model evaluation process here
        # For example:
        model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(224, (3, 3), activation='relu', input_shape=(224, 224, 3)),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10, activation='softmax')
        ])
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        
        # Load parameters
        model.set_weights(parameters)
        
        # Load test image data
        test_images = []
        for image_path, label in zip(image_paths[:100], labels[:100]):  # Use a subset of images for evaluation
            img, _ = load_image_with_label(image_path, label)
            img_resized = resize_image(img)  # Resize image
            test_images.append(img_resized)
        
        # Convert list to NumPy array
        x_test = np.array(test_images)
        y_test = np.array(labels[:100])
        
        # Evaluate the model on the test data
        loss, accuracy = model.evaluate(x_test, y_test)
        
        # Return loss, number of samples, and accuracy
        return loss, len(x_test), {"accuracy": accuracy}

def client_fn(cid: str):
    """Create and return an instance of Flower `Client`."""
    return FlowerClient().to_client()

# Flower ClientApp
app = ClientApp(
    client_fn=client_fn,
)

# Legacy mode
if __name__ == "__main__":
    from flwr.client import start_client
    start_client(
        server_address="127.0.0.1:8890",
        client=FlowerClient().to_client(),
    )


INFO :      
INFO :      [RUN 0, ROUND ]
INFO :      Received: get_parameters message 44660af4-bd33-4216-954a-ce5142b5a3cd
INFO :      Sent reply
INFO :      
INFO :      [RUN 0, ROUND ]
INFO :      Received: train message ee6ce4da-36c8-4bb3-b970-5d9f61e2ad84
